In [1]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [4]:
o3d.visualization.draw_geometries([pcd.pcd])

In [5]:
csf = CsfGroundExtractor()
csf.set_data(pcd)
csf._process_data()

In [15]:
ran = RansacGroundExtractor()
ran.set_data(pcd)
ran._process_data()

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [10]:
ground = gpf.np_to_o3d(pcd.ground)

In [18]:
class DriPathSegment(BaseAlgorithm):
    def __init__(self, data):
        self.data = data
        self.ground = self.data.ground
        self.ground_o3d = self.np_to_o3d(self.ground)
        self.ground_nor = np.asarray(self.ground_o3d.normals)
        self.no_paves = np.array([0])
        self.index_all = np.array(range(len(self.ground)))

        driving_track_seed = self.seed_select(self.ground_o3d)
        driving_track = self.driving_path_generation(driving_track_seed, self.ground_o3d)
        driving_track_o3d = self.np_to_o3d(driving_track)
        driving_track = self.find_nearest_point(self.ground_o3d, driving_track_o3d)

        driving_track_o3d = self.ground_o3d.select_by_index(driving_track)
        self.point_neighbours = self.find_nearest_point(driving_track_o3d, self.ground_o3d)
        self.driving_track = np.asarray(driving_track_o3d.points)


        

    def _process_data(self):

        for i in range(len(self.driving_track)):
            slim = self.ground[self.point_neighbours[:, 0] == i]
            slim_index = self.index_all[self.point_neighbours[:, 0] == i]
            if len(slim) > 0:
                slim_nor = self.ground_nor[self.point_neighbours[:, 0] == i]
                slim_distance = np.sqrt(np.sum(np.power(slim - self.driving_track[i, :], 2), axis=1))
                p = slim_distance.argmin()
                p = slim_nor[p, :]
                a_jiao = np.fabs(np.dot(slim_nor, p))
                c = a_jiao <= self.data.cosine_threshold
                self.no_paves = np.concatenate((self.no_paves, slim_index[c]), axis=0)

        no_paves = np.delete(self.no_paves, 0, 0)
        self.data.paves_o3d = self.ground_o3d.select_by_index(no_paves, invert = True)
        paves  = self.ground_o3d.select_by_index(no_paves, invert = True)

        self.data.paves = self.euclidean_cluster(paves, tolerance=0.15)

    def euclidean_cluster(self,cloud, tolerance=0.2):
        seed = self.seed_select(cloud)
        no_paves = self.find_no_paves(cloud)
        paves = np.asarray(cloud.points)
        no_paves = self.find_index(paves, no_paves)
        kdtree = o3d.geometry.KDTreeFlann(cloud)
        paves = np.array(seed)
        while len(seed) > 0 :
            seed_now = seed[0]
            k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[seed_now], tolerance)
            if k == 1 :
                continue
            idx = np.array(idx)
            idx = idx[np.isin(idx, paves, invert=True)]
            idx = idx[np.isin(idx, no_paves, invert=True)]
            paves = np.append(paves, idx)
            seed = np.append(seed,idx)
            seed = np.delete(seed,[0])

        return paves

In [6]:
o3d.visualization.draw_geometries([ground])

In [7]:
reg = ReGrowSegment(pcd)


[Open3D INFO] Picked point #8106 (-7.8, -14., -1.6) to add in queue.
[Open3D INFO] Picked point #31617 (0.37, -12., -1.6) to add in queue.


In [8]:
reg._process_data()

In [15]:
dps = DriPathSegment2(pcd)


[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #17861 (-3.7, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #8198 (-4.4, -15., -1.6) to add in queue.


In [7]:

dps._process_data()

In [19]:
dri = DriPathSegment(pcd)


[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #9532 (-10., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #17310 (-6.2, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #21246 (-3.2, -12., -1.6) to add in queue.
[Open3D INFO] Picked point #58473 (3.2, -9.0, -1.5) to add in queue.
[Open3D INFO] Picked point #66814 (5.1, -4.1, -1.5) to add in queue.
[Open3D INFO] Picked point #67032 (5.5, 1.3, -1.5) to add in queue.
[Open3D INFO] Picked point #68850 (5.3, 5.9, -1.5) to add in queue.


In [20]:
dri._process_data()

[Open3D INFO] Picked point #17427 (-3.5, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #33810 (0.44, -12., -1.6) to add in queue.
[Open3D INFO] Picked point #47651 (2.1, -9.8, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [11]:
no_ground = gpf.np_to_o3d(pcd.no_ground)
no_paves = ground.select_by_index(pcd.no_paves,invert = True)

In [ ]:
paves = pcd.paves_o3d.select_by_index(pcd.paves)
o3d.visualization.draw_geometries([paves])

In [25]:
paves.paint_uniform_color([1,0,0])

PointCloud with 60751 points.

In [26]:
o3d.visualization.draw_geometries([no_paves,paves,no_ground])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [12]:
paves  = ground.select_by_index(pcd.no_paves, invert = True)

In [13]:
o3d.visualization.draw_geometries([paves])

In [14]:
def euclidean_cluster(dps,cloud, tolerance=0.2):
    seed = dps.seed_select(cloud)
    no_paves = dps.find_no_paves(cloud)
    paves = np.asarray(cloud.points)
    no_paves = dps.find_index(paves, no_paves)
    kdtree = o3d.geometry.KDTreeFlann(cloud)
    paves = np.array(seed)
    while len(seed) > 0 :
        seed_now = seed[0]
        k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[seed_now], tolerance)
        if k == 1 :
            continue
        idx = np.array(idx)
        idx = idx[np.isin(idx, paves, invert=True)]
        idx = idx[np.isin(idx, no_paves, invert=True)]
        paves = np.append(paves, idx)
        seed = np.append(seed,idx)
        seed = np.delete(seed,[0])

    return paves

In [15]:
pave = euclidean_cluster(dri,paves, tolerance=0.15)

[Open3D INFO] Picked point #7260 (-8.9, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #18021 (-4.2, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #30229 (-2.2, -12., -1.6) to add in queue.


In [16]:
ground1 = paves.select_by_index(pave)
o3d.visualization.draw_geometries([ground1])